In [2]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
from string import punctuation
import numpy as np
import pandas as pd
import random
import sys
import io
import requests
import re

In [3]:
features = pd.read_csv("final_df2.csv")
features.head()

,Unnamed: 0,movie,release date,rating,genre,language,directors,duration,Box Office,Description,Audience Score,Tomatometer,Reviews,compound_audience,neutral_audience,positive_audience,negative_audience,positive,negative,neutral,compound,aud_asp_neutral,aud_asp_negative,aud_asp_positive,critic_reviews
0,0,chicken_run,"Jun 21, 2000",G,"['animation', 'kids and family', 'comedy', 'ad...",English,['Peter Lord'],1h 25m,NaN,"This engaging stop-motion, claymation adventur...",65.0,97.0,['While Chicken Run has a great animation styl...,0.395363,0.764375,0.208500,0.027125,0.14000,0.0675,0.79250,0.130605,0.127229,0.261164,0.611607,"Chicken Run, if anything, offers an alternativ..."
1,1,the_taste_of_others,"Feb 26, 2002",R,['comedy'],French (France),['Agnès Jaoui'],1h 52m,$635.3K,Castella (Jean-Pierre Bacri) is a successful b...,82.0,100.0,['A nice comedy that has pokes some friendly b...,0.544600,0.700125,0.237250,0.062625,0.25505,0.0301,0.71490,0.400550,0.044488,0.232734,0.722778,Succeeds with believable characters and situat...
2,2,yi_yi,"May 8, 2001",NaN,['drama'],Edward Yang,['Edward Yang'],2h 53m,$969.1K,"Set in Taiwan, the film follows the lives of t...",91.0,97.0,['A city symphony in minor key with a city-siz...,0.559800,0.806111,0.152556,0.041444,0.20600,0.0403,0.75370,0.348930,0.169338,0.233988,0.596674,"I felt like I had lived it, and not just becau..."
3,3,memento,"Mar 1, 2017",R (Language|Drug Content|Violence),['mystery and thriller'],English,['Christopher Nolan'],1h 56m,$25.5M,Leonard (Guy Pearce) is tracking down the man ...,94.0,93.0,"['Not a masterpiece but really good', ""Memento...",0.326714,0.740143,0.205429,0.054571,0.10860,0.0562,0.83515,0.157130,0.204062,0.217740,0.578198,Think of all the ways that a linear film struc...
4,4,the_life_and_times_of_hank_greenberg,"Jul 2, 1999",PG (Mild Language|Thematic Elements),"['biography', 'documentary']",English,['Aviva Kempner'],1h 29m,$1.7M,"""The Life and Times of Hank Greenberg"" is a hu...",80.0,97.0,"[""Dedicated to the millions of Detroit Tigers ...",0.840233,0.697333,0.289333,0.013333,0.22375,0.0243,0.75195,0.574785,0.047392,0.034910,0.917698,It's an adroit manipulation of interviews and ...


In [4]:
genre="documentary"
features=features.dropna(subset=["genre"])
selected_genre=features[features["genre"].str.contains(genre)]
selected_genre.head()

,Unnamed: 0,movie,release date,rating,genre,language,directors,duration,Box Office,Description,Audience Score,Tomatometer,Reviews,compound_audience,neutral_audience,positive_audience,negative_audience,positive,negative,neutral,compound,aud_asp_neutral,aud_asp_negative,aud_asp_positive,critic_reviews
4,4,the_life_and_times_of_hank_greenberg,"Jul 2, 1999",PG (Mild Language|Thematic Elements),"['biography', 'documentary']",English,['Aviva Kempner'],1h 29m,$1.7M,"""The Life and Times of Hank Greenberg"" is a hu...",80.0,97.0,"[""Dedicated to the millions of Detroit Tigers ...",0.840233,0.697333,0.289333,0.013333,0.223750,0.02430,0.751950,0.574785,0.047392,0.034910,0.917698,It's an adroit manipulation of interviews and ...
11,11,dark_days,"Dec 4, 2015",NaN,['documentary'],Marc Singer,['Marc Singer'],1h 34m,$436.8K,"""Dark Days"" is a feature length documentary ab...",89.0,94.0,['Equal parts inspiring and depressing. A lite...,0.906725,0.703750,0.221500,0.075000,0.090550,0.10805,0.801450,-0.006655,0.159434,0.424432,0.416135,This is the world discovered and illuminated b...
12,12,filth_and_the_fury,"Mar 29, 2000",R (Strong Language|Sexual Content|Drug Use),['documentary'],English,['Julien Temple'],1h 48m,$606.6K,"Assembled from unseen archive footage, rare of...",88.0,95.0,"[""Nevermind the Bollocks Here's the Sex Pistol...",-0.026756,0.745000,0.143222,0.111778,0.034091,0.00000,0.965909,0.057900,0.130965,0.411622,0.457413,"[]Best for older teens, especially Sex Pistols..."
20,20,wattstax,"Feb 18, 2016",R,"['music', 'documentary']",English,['Mel Stuart'],1h 38m,NaN,"In 1972 at the Los Angeles Coliseum, director ...",90.0,90.0,"['Black people of america, I Love You', 'Conju...",0.666143,0.800000,0.181714,0.018286,0.172250,0.03780,0.789950,0.423465,0.097406,0.077102,0.825492,All [the performers] draw lively reactions fro...
35,35,original_kings_of_comedy,"Aug 18, 2000",R (Language and Sex Related Humor),"['comedy', 'documentary']",English,['Spike Lee'],1h 55m,$38.2M,A cultural phenomenon for our time: Spike Lee ...,79.0,83.0,"[""Though an appealing lineup, the capturing do...",0.606740,0.649400,0.300200,0.050400,0.144250,0.05435,0.801400,0.317675,0.155835,0.421047,0.423119,[Spike Lee] knows the show belongs to his come...


In [5]:
combined = ' '.join(selected_genre["critic_reviews"])

In [7]:
processed_text = combined.lower()
processed_text=processed_text.translate(str.maketrans("", "", punctuation))
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text) 

In [8]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 223041
total chars: 37


In [9]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 74334


In [11]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [12]:
x.shape

(74334, 40, 37)

In [13]:
y.shape

(74334, 37)

In [14]:
y[0:10]

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, 

In [15]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

#optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

Build model...


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               84992     
_________________________________________________________________
dense (Dense)                (None, 37)                4773      
Total params: 89,765
Trainable params: 89,765
Non-trainable params: 0
_________________________________________________________________


In [17]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [18]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [19]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
model.fit(x, y,
          batch_size=128,
          epochs=50)
          #callbacks=[print_callback])

Epoch 1/100
581/581 [==============================] - 66s 110ms/step - loss: 2.2132
Epoch 2/100
581/581 [==============================] - 64s 110ms/step - loss: 1.8473
Epoch 3/100
581/581 [==============================] - 63s 109ms/step - loss: 1.6944
Epoch 4/100
581/581 [==============================] - 64s 110ms/step - loss: 1.5945
Epoch 5/100
581/581 [==============================] - 64s 110ms/step - loss: 1.5252
Epoch 6/100
581/581 [==============================] - 64s 110ms/step - loss: 1.4730
Epoch 7/100
581/581 [==============================] - 64s 110ms/step - loss: 1.4336
Epoch 8/100
581/581 [==============================] - 64s 110ms/step - loss: 1.4001
Epoch 9/100
581/581 [==============================] - 64s 110ms/step - loss: 1.3723
Epoch 10/100
581/581 [==============================] - 64s 110ms/step - loss: 1.3492
Epoch 11/100
581/581 [==============================] - 64s 110ms/step - loss: 1.3313
Epoch 12/100
581/581 [==============================] - 64s 110

In [20]:
path = f"{genre}.h5"
model.save_weights(path)

In [23]:
start_index = random.randint(0, len(processed_text) - maxlen - 1)
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('----- temperature:', temperature)

    generated = ''
    sentence = processed_text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

----- temperature: 0.2
----- Generating with seed: "ample joys and sorrows and it will intro"
ample joys and sorrows and it will introduzed to the subject who shays with the film is a strong it is a sining to be difference and the film is a strong chardagances its art too strendly the film is a truth is a strengs but it is a strike the soundtroves and the film is a strip and characters and the portrait of the film is a strong its all the way with the film is a strong is a collicial pressive film chrickand in the film is a strong
----- temperature: 0.5
----- Generating with seed: "ample joys and sorrows and it will intro"
ample joys and sorrows and it will introming in on the movie is a rousively funny portrait of the confide the film disying the reality of nonceity of a music fores a strong its all the uniquencely briews stunnently heart in the political gues a frespican lays in nothing to entertaining and had a bigk are concerting for the music funny his creat of the sort of a ballet 